In [ ]:
import os
import warnings

warnings.filterwarnings(action='ignore', message='numpy.dtype size changed')
warnings.filterwarnings(action='ignore', message='compiletime version 3.5 of module')

if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
os.chdir(os.path.split(workbookDir)[0])

%load_ext autoreload
%autoreload 2
%matplotlib inline


In [ ]:
from synthesized.testing.evaluation import Evaluation
from synthesized.testing.synthetic_distributions import *
evaluation = Evaluation()

### Bernoulli

In [ ]:
data = create_bernoulli(probability=0.2, size=10000)
synthesize_and_plot(data, name='bernoulli', evaluation=evaluation)

### Categorical

In [ ]:
data = create_categorical(probabilities=[0.5, 0.25, 0.125, 0.0625, 0.0625], size=10000)
synthesize_and_plot(data, name='categorical', evaluation=evaluation)

### 1-d Gaussian

In [ ]:
data = create_1d_gaussian(mean=1000,  std=100, size=10000)
synthesize_and_plot(data, name='1d_gaussian', evaluation=evaluation)